In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
ex1 = pd.read_parquet('train/8.성과정보/2018_성과정보_전체.parquet')

In [3]:
member_df = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')
member_seg = member_df[['ID', 'Segment']].drop_duplicates(subset='ID')
ex1 = ex1.merge(member_seg, on='ID', how='left')

In [4]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '증감율_이용건수_신용_전월', '증감율_이용건수_신판_전월', '증감율_이용건수_일시불_전월', '증감율_이용건수_할부_전월', '증감율_이용건수_CA_전월', '증감율_이용건수_체크_전월', '증감율_이용건수_카드론_전월', '증감율_이용금액_신용_전월', '증감율_이용금액_신판_전월', '증감율_이용금액_일시불_전월', '증감율_이용금액_할부_전월', '증감율_이용금액_CA_전월', '증감율_이용금액_체크_전월', '증감율_이용금액_카드론_전월', '증감율_이용건수_신용_분기', '증감율_이용건수_신판_분기', '증감율_이용건수_일시불_분기', '증감율_이용건수_할부_분기', '증감율_이용건수_CA_분기', '증감율_이용건수_체크_분기', '증감율_이용건수_카드론_분기', '증감율_이용금액_신용_분기', '증감율_이용금액_신판_분기', '증감율_이용금액_일시불_분기', '증감율_이용금액_할부_분기', '증감율_이용금액_CA_분기', '증감율_이용금액_체크_분기', '증감율_이용금액_카드론_분기', '잔액_신판평균한도소진율_r6m', '잔액_신판최대한도소진율_r6m', '잔액_신판평균한도소진율_r3m', '잔액_신판최대한도소진율_r3m', '잔액_신판ca평균한도소진율_r6m', '잔액_신판ca최대한도소진율_r6m', '잔액_신판ca평균한도소진율_r3m', '잔액_신판ca최대한도소진율_r3m', '변동률_일시불평잔', '변동률_RV일시불평잔', '변동률_할부평잔', '변동률_CA평잔', '변동률_RVCA평잔', '변동률_카드론평잔', '변동률_잔액_B1M', '변동률_잔액_일시불_B1M', '변동률_잔액_CA_B1M', '혜택수혜율_R3M', '혜택수혜율_B0M', 'Segment']

문자형 컬럼 이름:
['ID', 'Segment']

숫자형 컬럼 이름:
['기준년월', '증감율_이용건수_신용_전월', '증감율_이용건수_신판_전월', '증감율_이용건수_일시불_전월', '증감율_이용건수_할부_전월', '증감율_이

In [5]:
ex1 = ex1.fillna(-1)

In [6]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['증감율_이용건수_체크_전월', '증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '변동률_잔액_CA_B1M']

In [7]:
cols_to_drop = ['증감율_이용건수_체크_전월', '증감율_이용건수_카드론_전월', '증감율_이용금액_카드론_전월', '변동률_잔액_CA_B1M']

ex1.drop(columns=cols_to_drop, inplace=True)

In [8]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['증감율_이용건수_CA_전월', '증감율_이용금액_체크_전월', '증감율_이용건수_CA_분기', '증감율_이용금액_할부_전월', '변동률_CA평잔', '증감율_이용건수_체크_분기', '증감율_이용금액_CA_분기', '혜택수혜율_R3M', '증감율_이용금액_카드론_분기', '변동률_RVCA평잔', '혜택수혜율_B0M', '증감율_이용금액_CA_전월', '증감율_이용건수_카드론_분기', '변동률_카드론평잔', '증감율_이용건수_할부_전월', '증감율_이용금액_체크_분기']


In [9]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)

high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 51


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,잔액_신판평균한도소진율_r3m,잔액_신판최대한도소진율_r3m,0.994910,0.189758,0.194820
1,잔액_신판ca평균한도소진율_r3m,잔액_신판ca최대한도소진율_r3m,0.994069,0.258636,0.263688
2,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,0.988043,0.004801,0.005583
3,증감율_이용건수_신용_분기,증감율_이용건수_신판_분기,0.986543,0.000929,0.000335
4,잔액_신판ca평균한도소진율_r6m,잔액_신판ca최대한도소진율_r6m,0.986143,0.252525,0.268486
5,잔액_신판평균한도소진율_r6m,잔액_신판최대한도소진율_r6m,0.985459,0.181995,0.200087
6,증감율_이용금액_신용_분기,증감율_이용금액_신판_분기,0.983909,0.014716,0.013519
7,잔액_신판평균한도소진율_r6m,잔액_신판평균한도소진율_r3m,0.983587,0.181995,0.189758
8,잔액_신판최대한도소진율_r6m,잔액_신판최대한도소진율_r3m,0.981763,0.200087,0.194820
9,잔액_신판ca최대한도소진율_r6m,잔액_신판ca최대한도소진율_r3m,0.979922,0.268486,0.263688


In [10]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))

print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 18
제거할 피처 목록:
['증감율_이용건수_일시불_분기', '증감율_이용금액_일시불_분기', '잔액_신판최대한도소진율_r6m', '잔액_신판평균한도소진율_r3m', '증감율_이용금액_할부_분기', '잔액_신판ca평균한도소진율_r6m', '증감율_이용건수_신판_전월', '잔액_신판최대한도소진율_r3m', '증감율_이용건수_신판_분기', '증감율_이용금액_신용_전월', '잔액_신판ca평균한도소진율_r3m', '잔액_신판평균한도소진율_r6m', '변동률_잔액_B1M', '증감율_이용건수_신용_분기', '증감율_이용금액_신판_전월', '증감율_이용금액_신판_분기', '증감율_이용건수_신용_전월', '잔액_신판ca최대한도소진율_r3m']


In [11]:
cols_to_drop = ['증감율_이용건수_일시불_분기', '증감율_이용금액_일시불_분기', '잔액_신판최대한도소진율_r6m', '잔액_신판평균한도소진율_r3m', '증감율_이용금액_할부_분기', '잔액_신판ca평균한도소진율_r6m', '증감율_이용건수_신판_전월', '잔액_신판최대한도소진율_r3m', '증감율_이용건수_신판_분기', '증감율_이용금액_신용_전월', '잔액_신판ca평균한도소진율_r3m', '잔액_신판평균한도소진율_r6m', '변동률_잔액_B1M', '증감율_이용건수_신용_분기', '증감율_이용금액_신판_전월', '증감율_이용금액_신판_분기', '증감율_이용건수_신용_전월', '잔액_신판ca최대한도소진율_r3m']
ex1.drop(columns=cols_to_drop, inplace=True)

In [12]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [13]:
ex1

,기준년월,ID,증감율_이용건수_일시불_전월,증감율_이용금액_일시불_전월,증감율_이용건수_할부_분기,증감율_이용금액_신용_분기,잔액_신판ca최대한도소진율_r6m,변동률_일시불평잔,변동률_RV일시불평잔,변동률_할부평잔,변동률_잔액_일시불_B1M,Segment,Segment_encoded
0,201807,TRAIN_000000,0.364727,0.313300,0.000000,-0.411320,0.849842,0.951930,0.999998,1.042805,0.270752,D,3
1,201807,TRAIN_000001,-0.511808,-1.010817,0.000000,0.552668,0.851009,1.290439,1.092698,0.905663,-0.670348,E,4
2,201807,TRAIN_000002,0.067843,0.289565,1.999996,0.139121,0.938161,0.929196,1.006124,1.993590,0.058114,C,2
3,201807,TRAIN_000003,0.153975,0.276947,0.000000,0.103217,1.135424,1.020964,0.999998,1.050646,0.258943,D,3
4,201807,TRAIN_000004,0.000000,0.000028,0.000000,-1.999996,0.000000,0.000000,0.999998,0.999998,0.000000,E,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0.000000,0.000027,0.000000,0.000000,0.032439,0.999998,0.999998,0.999998,0.000000,E,4
2399996,201812,TRAIN_399996,0.073804,0.207099,0.000000,-0.792478,0.168081,0.615087,0.999998,0.999998,-0.159143,D,3
2399997,201812,TRAIN_399997,-0.040432,0.229573,1.999996,0.042752,0.190393,1.036370,0.999998,0.345027,0.126581,C,2
2399998,201812,TRAIN_399998,0.000000,0.000034,0.000000,0.000000,0.012677,0.999998,0.999998,0.999998,0.000000,E,4


In [14]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [15]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월', 'ID', 'Segment', '잔액_신판ca최대한도소진율_r6m', '변동률_RV일시불평잔']

In [16]:
ex1_filtered.to_parquet('성과_전처리_Segment.parquet', index=False)